### 1. What sequences were used for analysis? Provide type of biopolymer, organism, gene name.

Nucleotide sequences of SUP35 (Systematic Name YDR172W) protein from form Yeast.

In [1]:
!head -n 2 data_part2/SUP35_10seqs.fa

>SUP35_Kla_AB039749
ATGTCAGACCAACAAAATCAAGACCAAGGGCAAGGCCAAGGTTACAATCAGTATAACCAATATGGCCAGT


### 2. Run at least 4 different alignment algorithms. Create comparison table with running time* and comments on the DNA alignment quality for the above algorithms**. Which algorithm is better to use?

In [2]:
!time muscle -in data_part2/SUP35_10seqs.fa -out data_part2/muscle_alignment.fa


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

SUP35_10seqs 10 seqs, lengths min 2001, max 2103, avg 2049












real	0m7.132s
user	0m6.198s
sys	0m0.146s


In [3]:
!time mafft data_part2/SUP35_10seqs.fa > data_part2/mafft_alignment.fa

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
    1 / 10
done.

Constructing a UPGMA tree (efffree=0) ... 
    0 / 10
done.

Progressive alignment 1/2... 
STEP     8 / 9  f
Reallocating..done. *alloclen = 5305
STEP     9 / 9  f
done.

Making a distance matrix from msa.. 
    0 / 10
done.

Constructing a UPGMA tree (efffree=1) ... 
    0 / 10
done.

Progressive alignment 2/2... 
STEP     8 / 9  f
Reallocating..done. *alloclen = 5302
STEP     9 / 9  f
done.

disttbfast (nuc) Version 7.455
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scori

In [4]:
!time kalign data_part2/SUP35_10seqs.fa data_part2/kalign_alignment.fa

EXTRA :2

Kalign version 2.04, Copyright (C) 2004, 2005, 2006 Timo Lassmann

        Kalign is free software. You can redistribute it and/or modify
        it under the terms of the GNU General Public License as
        published by the Free Software Foundation.

reading from STDIN: found no sequences.

reading from data_part2/SUP35_10seqs.fa: found 10 sequences
reading from data_part2/kalign_alignment.fa: found no sequences.
-> output file, in fasta format

Aligning 10 RNA/DNA sequences with these parameters:
        217.00000000	gap open penalty
        39.40000153	gap extension
        292.60000610	terminal gap penalty
        283.00000000	bonus
Alignment will be written to file:'data_part2/kalign_alignment.fa'.

Distance Calculation:
     100 percent done
Alignment:
       0 percent doneSaving mem...
      10 percent doneSaving mem...
      20 percent doneSaving mem...
      30 percent doneSaving mem...
      40 percent doneSaving mem...
      50 percent doneSaving mem...
      60 

In [5]:
!time prank -o=data_part2/prank_alignment.fa -d=data_part2/SUP35_10seqs.fa


-----------------
 PRANK v.170427:
-----------------

Input for the analysis
 - aligning sequences in 'data_part2/SUP35_10seqs.fa'
 - using inferred alignment guide tree
 - option '+F' is not used; it can be enabled with '+F'
 - external tools available:
    MAFFT for initial alignment

Correcting (arbitrarily) for multifurcating nodes.
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 1.

Alignment score: 4527
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 2.

Alignment score: 4513
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 3.

Alignment score: 4513
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 4.

Alignment score: 4513
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 5.

Alignment score: 4513


Writing
 - alignment to 'data_part2/prank_alignment.fa.best.fas

In [6]:
from Bio import AlignIO, SeqIO

import numpy as np
import pandas as pd

import scipy.stats

In [7]:
def get_alignment_length_and_entropy(file):
    for multiple_alignment in AlignIO.parse(file, 'fasta'):
        entropy = 0
        sequences = [a.seq for a in multiple_alignment]
        for aligned_letter in zip(*sequences):
            ln = len(aligned_letter)
            fs = [f / ln for f in np.unique(aligned_letter, return_counts=True)[1]]
            entropy += scipy.stats.entropy(fs)
        return multiple_alignment.get_alignment_length(), entropy
    
methods = ['muscle', 'mafft', 'kalign', 'prank']
time = {
    'muscle': 7.132,
    'mafft': 1.336,
    'kalign': 2.607,
    'prank': 162.251
}


res = []
for method in methods:
    length, entropy = get_alignment_length_and_entropy(f'data_part2/{method}_alignment.fa')
    res.append([method, length, entropy, time[method]])
    
pd.DataFrame(res, columns=['Method', 'Alignment Length', 'Alignment Entropy', 'Elapsed Time'])

,Method,Alignment Length,Alignment Entropy,Elapsed Time
0,muscle,2275,688.028202,7.132
1,mafft,2286,707.579148,1.336
2,kalign,2155,653.044547,2.607
3,prank,2366,719.030552,162.251


### Results

Prank is seemed to be the worst algorithm, it took a lot of time to compute and got the worst results.
The best score and good performance were shown by kalign.
Mafft was the fastest one.


### 3. What is wrong with the alignment of SUP35_10seqs_strange_aln.fa and how to fix it?

The gap size is not always multiple of 3. Also, it contains too many gaps and too much length. 

### 4.Obtain amino acid sequences for these data. Repeat p.2 on amino acid sequences.

In [8]:
translated = [cnt.translate() for cnt in SeqIO.parse('data_part2/SUP35_10seqs.fa', 'fasta')]
SeqIO.write(translated, 'data_part2/protein_SUP35_10seqs.fa', 'fasta')

10

In [9]:
!time muscle -in data_part2/protein_SUP35_10seqs.fa -out data_part2/protein_muscle_alignment.fa


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.


*** WARNING *** Invalid character '*' in FASTA sequence data, ignored

*** WARNING *** Invalid character '*' in FASTA sequence data, ignored

*** WARNING *** Invalid character '*' in FASTA sequence data, ignored

*** WARNING *** Invalid character '*' in FASTA sequence data, ignored

*** WARNING *** Invalid character '*' in FASTA sequence data, ignored

*** WARNING *** Invalid character '*' in FASTA sequence data, ignored

*** WARNING *** Invalid character '*' in FASTA sequence data, ignored

*** WARNING *** Invalid character '*' in FASTA sequence data, ignored

*** WARNING *** Invalid character '*' in FASTA sequence data, ignored

*** WARNING *** Invalid character '*' in FASTA sequence data, ignored
protein_SUP35_10seqs 10 seqs, lengths min 666, max 700, avg 682



































real	0m0.581s
use

In [10]:
!time mafft data_part2/protein_SUP35_10seqs.fa > data_part2/protein_mafft_alignment.fa

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
    1 / 10
done.

Constructing a UPGMA tree (efffree=0) ... 
    0 / 10
done.

Progressive alignment 1/2... 
STEP     8 / 9  f
Reallocating..done. *alloclen = 2436
STEP     9 / 9  f
done.

Making a distance matrix from msa.. 
    0 / 10
done.

Constructing a UPGMA tree (efffree=1) ... 
    0 / 10
done.

Progressive alignment 2/2... 
STEP     8 / 9  f
Reallocating..done. *alloclen = 2436
STEP     9 / 9  f
done.

disttbfast (aa) Version 7.455
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It ten

In [11]:
!time kalign data_part2/protein_SUP35_10seqs.fa data_part2/protein_kalign_alignment.fa

EXTRA :2

Kalign version 2.04, Copyright (C) 2004, 2005, 2006 Timo Lassmann

        Kalign is free software. You can redistribute it and/or modify
        it under the terms of the GNU General Public License as
        published by the Free Software Foundation.

reading from STDIN: found no sequences.

reading from data_part2/protein_SUP35_10seqs.fa: found 10 sequences
reading from data_part2/protein_kalign_alignment.fa: found no sequences.
-> output file, in fasta format

Aligning 10 protein sequences with these parameters:
        54.94940948	gap open penalty
        8.52492046	gap extension
        4.42409992	terminal gap penalty
        0.20000000	bonus
Alignment will be written to file:'data_part2/protein_kalign_alignment.fa'.

Distance Calculation:
     100 percent done
Alignment:
     100 percent done

real	0m0.644s
user	0m0.280s
sys	0m0.019s


In [12]:
!time prank -o=data_part2/protein_prank_alignment.fa -d=data_part2/protein_SUP35_10seqs.fa


-----------------
 PRANK v.170427:
-----------------

Input for the analysis
 - aligning sequences in 'data_part2/protein_SUP35_10seqs.fa'
 - using inferred alignment guide tree
 - option '+F' is not used; it can be enabled with '+F'
 - external tools available:
    MAFFT for initial alignment

Sequence name <unknown id> <unknown description> is defined more than once! Adding suffix '.1'.
Sequence name <unknown id> <unknown description> is defined more than once! Adding suffix '.1'.
Sequence name <unknown id> <unknown description>.1 is defined more than once! Adding suffix '.1'.
Sequence name <unknown id> <unknown description> is defined more than once! Adding suffix '.1'.
Sequence name <unknown id> <unknown description>.1 is defined more than once! Adding suffix '.1'.
Sequence name <unknown id> <unknown description>.1.1 is defined more than once! Adding suffix '.1'.
Sequence name <unknown id> <unknown description> is defined more than once! Adding suffix '.1'.
Sequence name <unknown 

In [14]:
time = {
    'muscle': 0.581,
    'mafft': 0.985,
    'kalign': 0.644,
    'prank': 248.340
}


res = []
for method in methods:
    length, entropy = get_alignment_length_and_entropy(f'data_part2/protein_{method}_alignment.fa')
    res.append([method, length, entropy, time[method]])
    
pd.DataFrame(res, columns=['Method', 'Alignment Length', 'Alignment Entropy', 'Elapsed Time'])



,Method,Alignment Length,Alignment Entropy,Elapsed Time
0,muscle,743,188.239341,0.581
1,mafft,759,193.104041,0.985
2,kalign,720,192.539250,0.644
3,prank,777,209.410913,248.340


The difference in entropy has become smaller, which is due to less information in amino acids than in nucleotides.